In [1]:
class GradientArtefactCorrection():

    def __init__(self):
        pass

    def fit_transform(self, raw, event_name, half_window_size=10):

        tr_events, event_id = mne.events_from_annotations(raw, event_id='auto', regexp=event_name, use_rounding=True, chunk_duration=None, verbose=None)

        tr = np.diff(tr_events[:,0])
        tr = np.mean(tr).astype(int)
        tr_s = tr / raw.info['sfreq']

        print(tr_s)

        epochs = mne.Epochs(raw, tr_events, event_id, tmin=0, tmax=tr_s, baseline=None, preload=False).drop_bad()
        epochs_detrended = mne.Epochs(raw, tr_events, event_id, tmin=0, tmax=tr_s, baseline=None, preload=False, detrend=0).drop_bad()
        
        corrected = raw.get_data()
        for t,tr_event in enumerate(tr_events[:,0]):
            start = tr_event
            stop = start + tr + 1
            if stop > raw.n_times:
                stop = raw.n_times

            indice_min = t - half_window_size 
            indice_max = t + half_window_size + 1

            if t < half_window_size:
                indice_min = 0
                indice_max = 2 * half_window_size + 1

            if t > len(epochs) - half_window_size:
                indice_max = len(epochs)
                indice_min = len(epochs) - 2 * half_window_size - 1

            print(indice_min, indice_max)
            template = epochs_detrended[indice_min:indice_max].average().data
            data = epochs[t].get_data()[0]
            corrected[:, start:stop] = data - template

        raw_corrected = mne.io.RawArray(corrected, raw.info)
        raw_corrected.set_annotations(raw.annotations)
        return(raw_corrected)

GAC = GradientArtefactCorrection()
raw_corrected = GAC.fit_transform(raw, event_name='Response/R128', half_window_size=10)   

NameError: name 'raw' is not defined

In [95]:
raw_corrected.plot()

In [103]:
raw.get_data().shape

(36, 1325900)

In [101]:
raw.get_data()[0,0:raw.n_times].shape

(1325900,)

In [2]:
raw = mne.io.read_raw_brainvision(r'C:\Users\Victor\Documents\GitHub\neuxus_test\P05_eyes_open_mrion.vhdr', preload=True)

NameError: name 'mne' is not defined

In [18]:
data = raw.get_data()
tr_events = []
for annotation in raw.annotations:
    if annotation['description'] == 'Response/R128':
        tr_events.append(int(annotation['onset']  * raw.info['sfreq']))
tr_events = np.asarray(tr_events)

gr = GradientRemover(data, tr_events)
corrected = gr.corrected
raw_corrected = mne.io.RawArray(corrected, raw.info, verbose=False)

TypeError: only integer scalar arrays can be converted to a scalar index

In [15]:
raw_corrected.plot()

In [16]:
raw.plot()